## Install Required Packages

In [ ]:
# %pip install --upgrade raiwidgets
# %pip install --upgrade pandas
# %pip install --upgrade fairlearn
# %pip install --upgrade interpret-community

After installing packages, you must close and reopen the notebook as well as restarting the kernel.

## Error Analysis

**Error Analysis** powered by [Error Analysis](https://github.com/microsoft/responsible-ai-toolbox/blob/main/docs/erroranalysis-dashboard-README.md), which identifies cohorts of data with higher error rate than the overall benchmark. These discrepancies might occur when the system or model underperforms for specific demographic groups or infrequently observed input conditions in the training data.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from responsibleai import RAIInsights
from raiwidgets import ResponsibleAIDashboard

In [ ]:
# Load the dataset
data = pd.read_csv('/home/josh/Downloads/AiDashOwnModels/Acidents_dataset.csv')

# Check column names for verification
print("Column names in the dataset:", data.columns)

# Define target column
target_column = 'Degree of Injury'

In [ ]:
# Ensure target column exists
if target_column in data.columns:
    print(f"Target column '{target_column}' exists.")
else:
    print(f"Target column '{target_column}' does NOT exist. Check column name.")

# Separate features (X) and target (y)
X = data.drop(columns=[target_column])  # Drop target column from features
y = data[target_column]  # Target column (keep in original form)

# Handle categorical variables by converting to numeric (one-hot encoding)
X = pd.get_dummies(X, drop_first=True)  # Convert categorical variables to one-hot encoding

# Ensure all feature columns are numeric
X = X.astype(float)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForestClassifier model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [ ]:
preds = model.predict(X_test)
print("Unique predictions from model:", pd.Series(preds).unique())

# Combine training and test features with their corresponding targets for RAIInsights
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

In [ ]:
# Create RAIInsights object
rai_insights = RAIInsights(model=model,
                           train=train_data,
                           test=test_data,
                           target_column=target_column,
                           task_type='classification')

# Pass the desired class as a list to ensure proper recognition


# Add Counterfactuals, Explainers, and Error Analysis
rai_insights.counterfactual.add(total_CFs=10, desired_class='opposite')
rai_insights.explainer.add()
rai_insights.error_analysis.add()

In [ ]:
# Compute insights
rai_insights.compute()

In [ ]:
ResponsibleAIDashboard(rai_insights)

## © Copyright, 2025 Assentian Limited. All Rights Reserved 